In [25]:
import plotly.graph_objs as go
import random
import numpy as np
import torch
from differentiable_DR_model import DRagent
from baseline import MLP, RNNmodel
import keras

In [29]:
df_dp = np.load(f"dataset/co50/data_N30_0.npz")
df_dp['paras']

array([[0.5 , 0.5 , 3.  , 0.94]])

In [23]:
dataset = np.load(f"../EnergyStorage/dataset/data_N365_1.npz")
P1 = dataset["p"].max()
P2 = dataset["d"].max()
T = 24
model = DRagent(P1, P2, T)
model.load_state_dict(torch.load('../EnergyStorage/result/model_gradient/model_1.pth'))
model.eval()

DRagent(
  (costNN): ICNN(
    (linear_z1): Linear(in_features=24, out_features=24, bias=False)
    (linear_y0): Linear(in_features=1, out_features=24, bias=True)
    (linear_y1): Linear(in_features=1, out_features=1, bias=True)
    (act): Softplus(beta=1, threshold=20)
  )
  (layer): OptLayer()
)

In [24]:
model.eta

Parameter containing:
tensor([0.8558], requires_grad=True)

# plot validation loss

In [48]:
val_loss_mlp = np.zeros((10, 500))
val_loss_rnn = np.zeros((10, 500))
val_loss_model = np.zeros((10, 500))
for i in range(10):
    val_loss_mlp[i] = np.load(f'result/baseline/MLP_loss_{i}.npz')['val_loss'][:500]
    val_loss_rnn[i] = np.load(f'result/baseline/RNN_loss_{i}.npz')['val_loss'][:500]
for i in range(10):
    val_loss_model[i] = np.load(f'result/model_gradient/ICNN/loss_{i}.npz')['val_loss']

In [49]:
fig = go.Figure()
x1 = np.linspace(0,500,100)
x2 = np.linspace(0,500,500)

median_NN =  np.median(val_loss_mlp, axis=0)[:500]
median_NN_2 = np.quantile(val_loss_mlp,0.2, axis=0)[:500]
median_NN_8 = np.quantile(val_loss_mlp,0.8,axis=0)[:500]

median_RNN =  np.median(val_loss_rnn, axis=0)[:500]
median_RNN_2 = np.quantile(val_loss_rnn,0.2,axis=0)[:500]
median_RNN_8 = np.quantile(val_loss_rnn,0.8,axis=0)[:500]

median_opt1 =  np.median(val_loss_model, axis=0)[:500]
median_opt1_2 = np.quantile(val_loss_model, 0.2, axis=0)[:500]
median_opt1_8 = np.quantile(val_loss_model, 0.8, axis=0)[:500]


fig.add_trace(go.Scatter(x=x1, y=median_NN_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False ))
fig.add_trace(go.Scatter(x=x1,y=median_NN_8,fill='tonexty', mode='lines',  fillcolor='rgba(203,158,252,0.2)',line_color = 'rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x1, y=median_RNN_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False ))
fig.add_trace(go.Scatter(x=x1,y=median_RNN_8,fill='tonexty', mode='lines',  fillcolor='rgba(168,236,217,0.5)',line_color='rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x2, y=median_opt1_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False    ))
fig.add_trace(go.Scatter(x=x2,y=median_opt1_8,fill='tonexty', mode='lines',  fillcolor='rgba(240,96,72,0.5)',line_color = 'rgba(0,0,0,0)',showlegend = False))


fig.add_trace(go.Scatter(x=x1,y=median_NN, mode='lines',  line_color='#b681f0',opacity=0.5, line_width=3,name = 'median(MLP,300)')) 
fig.add_trace(go.Scatter(x=x1,y=median_RNN, mode='lines',  line_color='#07cd98',opacity=0.7,line_width=3,name = 'median(RNN,300)')) 
fig.add_trace(go.Scatter(x=x2,y=median_opt1, mode='lines',  line_color='#f06048',opacity=0.5,line_width=3,name = 'median(Ours,20)')) 

fig.update_layout({#'title': 'NRMSE comparison',
    'xaxis':{'title':'Iterations', 'showline':True},
    'yaxis':{'title': 'Validation Loss',"range":[0,0.2]},
    "font": {"size": 40, 'color':'#3a4142'},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':1600,
   'height':900
})

fig.show()


In [50]:
fig.write_image("result/image/validation_loss.pdf")

# Plot Prediction

In [2]:
rnnmodel = keras.models.load_model('result/baseline/model_RNN')
mlpmodel = keras.models.load_model('result/baseline/model_MLP')
dataset = np.load(f"dataset/data_N365_1.npz")
P1 = dataset["p"].max()
P2 = dataset["d"].max()
T = 24
model = DRagent(P1, P2, T)
model.load_state_dict(torch.load('result/model_gradient/ICNN/model_1.pth'))
model.eval()

2022-09-05 17:42:30.265533: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


DRagent(
  (costNN): ICNN(
    (linear_z1): Linear(in_features=24, out_features=24, bias=False)
    (linear_y0): Linear(in_features=1, out_features=24, bias=True)
    (linear_y1): Linear(in_features=1, out_features=1, bias=True)
    (act): Softplus(beta=1, threshold=20)
  )
  (layer): OptLayer()
)

In [4]:
test_price = dataset['price'][:10]
df_dp = dataset
d2 = df_dp["d"][:10]
p2 = df_dp["p"][:10]
test_dr = p2-d2
price_tensor2 = torch.from_numpy(test_price).double()
d_tensor2 = torch.from_numpy(d2).double()
p_tensor2 = torch.from_numpy(p2).double()
pred_d2, pred_p2 = model(price_tensor2, d_tensor2, 500)
pred_d2 = pred_d2.detach().numpy()
pred_p2 = pred_p2.detach().numpy()
pred_dr = pred_p2-pred_d2

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-27


/Users/bb/opt/anaconda3/envs/dr/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:178: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


In [36]:
i = np.random.randint(10)
dr1 = rnnmodel(test_price[i:i+1])
dr3 = mlpmodel(test_price[i:i+1])

t1 = go.Scatter(x=np.arange(24), y=test_price[i],mode='lines', name='price',line = {'width':6,'dash':'dash','color':'#ffa05a'},yaxis='y2')
t2 = go.Scatter(x=np.arange(24), y=test_dr[i],mode='lines', name='True',line = {'width':6,'dash':'dash'})
t3 = go.Scatter(x=np.arange(24), y=dr1[0],mode='lines', name='RNN(300)',line = {'width':4})
t5 = go.Scatter(x=np.arange(24), y=pred_dr[i],mode='lines', name='Ours',line = {'width':4})
t6 = go.Scatter(x=np.arange(24), y=dr3[0],mode='lines', name='MLP(300)',line = {'width':4})
fig = go.Figure(data=[t2,t5,t3,t6,t1],layout={
    'xaxis':{'title':'Hour', 'showline':True},
    'yaxis':{'title': 'Dispatch(MW)','range':[-0.8,0.5]},
    'yaxis2': {'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right','range':[10,100], 'showgrid':False},
    "font": {"size": 40,'color':'#3a4142'},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':1600,
    'height':900
})
fig.show()

In [14]:
fig.write_image("result/image/predict.pdf")